- 중요한 변수들을 변형하여 파생변수를 생성하였고, 몇몇 변수에 데이터 재표현을 사용하여 비선형 변환을 해주었습니다.
- 단일 알고리즘으로는 성능의 한계를 느껴서 스태킹을 시도했습니다.
- 부스팅과 배깅 알고리즘에 맞게 feature_set을 2개 만들어서 다양한 알고리즘에 넣어보았습니다.
- 가장 성능이 좋은 light gbm 알고리즘은 피라미터를 다르게 하여 2개 생성하였습니다.
- 파라미터 튜닝은 Grid Search CV와 경험에 의존하여 정하였습니다.

In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split,GridSearchCV
from tqdm import tqdm

from pathlib import Path
import warnings
warnings.filterwarnings(action = 'ignore')

### 데이터 불러오기

In [25]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

### feature_set 1

In [26]:
for data in [train, test]:
    
    ## 변수 변환
    data['birth_plus'] = data.DAYS_BIRTH*(-1)/365
    
    ## 라벨 인코딩
    data['gender']=LabelEncoder().fit_transform(data.gender)
    data['car']=LabelEncoder().fit_transform(data.car)
    data['reality']=LabelEncoder().fit_transform(data.reality)
    data['income_type']=LabelEncoder().fit_transform(data.income_type)
    data['edu_type']=LabelEncoder().fit_transform(data.edu_type)
    data['family_type']=LabelEncoder().fit_transform(data.family_type)
    data['house_type']=LabelEncoder().fit_transform(data.house_type)
    data.fillna('NAN',inplace=True)
    data['occyp_type']=LabelEncoder().fit_transform(data.occyp_type)
    data['income_weight']=data.income_total**2
    data['income_age']=data.income_total*train.DAYS_BIRTH
    data['income_emp']=data.income_total*train.DAYS_EMPLOYED
    
    ## type 변환
    data.income_total=data.income_total.astype(int)
    data.family_size=data.family_size.astype(int)
    data.begin_month=data.begin_month.astype(int)
    data.birth_plus=data.birth_plus.astype(int)

    ## 변수 제거
    data.drop(['index','family_size','work_phone'],axis=1,inplace=True)
    
    ## 파생 변수 생성
    data.DAYS_BIRTH=data.DAYS_BIRTH**6
    data.income_type=1/(data.income_type)
    data.income_weight=1/(data.income_weight)**3
    data['DAYS_BIRTH_weight']=1/(data.DAYS_BIRTH)**2

## catboost는 categorical로 두면 안 됨
cat_credit=train.credit.astype('int').copy()
train.credit=train.credit.astype('category')

## feature_set 1 저장
ftr1 = train.drop('credit', axis=1).values
target1 = train['credit'].values
tst_ar1 = test.values
n_class = 3

### feature_set 2

In [ ]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

## 365 변환
def days_to_plus(x):
    return (x*-1)/365

## 업무 시작일 변환
def plus(x):
    if x<0:
        return x*(-1)
    else:
        return 0

    
## 변수 변환 (업무 시작일과 나이)
for i in [trn, tst]:
    i['birth_plus'] = i.DAYS_BIRTH.map(days_to_plus)
    i['employed_plus'] = i.DAYS_EMPLOYED.map(days_to_plus)


## 라벨 인코딩
index_col = 'index'
target_col = 'credit'

cat_cols = [x for x in trn.columns if trn[x].dtype == 'object']
float_cols = [x for x in trn.columns.drop('credit') if trn[x].dtype == 'float64']
num_cols = [x for x in trn.columns if x not in cat_cols + [target_col]]
feature_cols = num_cols + cat_cols

lbe = LabelEncoder()
for i in cat_cols:
    trn[i] = lbe.fit_transform(trn[i].astype(str))
    tst[i] = lbe.transform(tst[i].astype(str))
    
for i in float_cols:
    trn[i] = trn[i].astype('int')
    tst[i] = tst[i].astype('int')
    

## 인덱스 설정 및 변수 제거
for i in [trn, tst]:
    i.set_index('index', inplace = True)
    i.drop(['family_size', 'employed_plus'], axis = 1, inplace = True)


## 파생변수 생성
for i in [trn,tst]:
    # 연봉 제곱 변수 제거
    i['income_total_2'] = i.income_total**2

    # DAYS_BIRTH 제곱 변수 추가
    i['DAYS_BIRTH**2'] = i.DAYS_BIRTH**2

    # income * days_birth
    i['income_age'] = i.income_total*i.DAYS_BIRTH

    # income * DAYS_EMPLOYED
    i['income_emp'] = i.income_total*i.DAYS_EMPLOYED


## 데이터 재표현 (비선형 변환)
for i in [trn, tst]:
    i['DAYS_BIRTH**2'] = np.log1p(i['DAYS_BIRTH**2'])
    i['income_total_2'] = i.income_total_2**2
    i['income_emp'] = i.income_emp**2
    i['income_age'] = i.income_age**3


## 변수 타입 전환
for i in [trn, tst]:
    i['income_total'] = i.income_total.astype(int)
    i['DAYS_BIRTH**2'] = i['DAYS_BIRTH**2'].astype(int)
trn.credit = trn.credit.astype('category')


## feature_set 2 저장
ftr2 = trn.drop('credit', axis=1).values
target2 = trn['credit'].values
tst_ar2 = tst.values
n_class = 3

### LGB_1

In [5]:
n_fold = 17
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr1.shape)

lgb_p_val1 = np.zeros((ftr1.shape[0], n_class))
lgb_p_tst1 = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    lgb_clf = LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 50)

    lgb_clf.fit(ftr1[i_trn], target1[i_trn],
            eval_set=[(ftr1[i_val], target1[i_val])])

    lgb_p_val1[i_val, :] = lgb_clf.predict_proba(ftr1[i_val])
    lgb_p_tst1 += lgb_clf.predict_proba(tst_ar1) / n_fold
    
print(f'{log_loss(target1, lgb_p_val1)}')
print(f'{confusion_matrix(target1, np.argmax(lgb_p_val1, axis=1))}%')

(26457, 21)
training model for CV #1
[1]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.874395
[2]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.869746
[3]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.865208
[4]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.858514
[5]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.85165
[6]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.844053
[7]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.83626
[8]	valid_0's multi_error: 0.356868	valid_0's multi_logloss: 0.834209
[9]	valid_0's multi_error: 0.354942	valid_0's multi_logloss: 0.829365
[10]	valid_0's multi_error: 0.353659	valid_0's multi_logloss: 0.826282
[11]	valid_0's multi_error: 0.350449	valid_0's multi_logloss: 0.821945
[12]	valid_0's multi_error: 0.347882	valid_0's multi_logloss: 0.817375
[13]	valid_0's multi_error: 0.338254	valid_0's multi_logloss: 0.813539
[14]	valid_0's multi_error: 0.33697	valid_0's multi

[230]	valid_0's multi_error: 0.265083	valid_0's multi_logloss: 0.672853
training model for CV #2
[1]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.874405
[2]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.871475
[3]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.867012
[4]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.859942
[5]	valid_0's multi_error: 0.358793	valid_0's multi_logloss: 0.853071
[6]	valid_0's multi_error: 0.35751	valid_0's multi_logloss: 0.846505
[7]	valid_0's multi_error: 0.35751	valid_0's multi_logloss: 0.839045
[8]	valid_0's multi_error: 0.35751	valid_0's multi_logloss: 0.837305
[9]	valid_0's multi_error: 0.356868	valid_0's multi_logloss: 0.832921
[10]	valid_0's multi_error: 0.356868	valid_0's multi_logloss: 0.830816
[11]	valid_0's multi_error: 0.354942	valid_0's multi_logloss: 0.826695
[12]	valid_0's multi_error: 0.348524	valid_0's multi_logloss: 0.822084
[13]	valid_0's multi_error: 0.33697	valid_0's multi_logloss: 0.

[229]	valid_0's multi_error: 0.270218	valid_0's multi_logloss: 0.706822
[230]	valid_0's multi_error: 0.270218	valid_0's multi_logloss: 0.70651
training model for CV #3
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.875551
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.872744
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.869089
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.862492
[5]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.856444
[6]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.850192
[7]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.843575
[8]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.841297
[9]	valid_0's multi_error: 0.356455	valid_0's multi_logloss: 0.836833
[10]	valid_0's multi_error: 0.356455	valid_0's multi_logloss: 0.834321
[11]	valid_0's multi_error: 0.353886	valid_0's multi_logloss: 0.830413
[12]	valid_0's multi_error: 0.34939	valid_0's multi_logloss:

[228]	valid_0's multi_error: 0.274888	valid_0's multi_logloss: 0.691217
[229]	valid_0's multi_error: 0.276172	valid_0's multi_logloss: 0.690994
[230]	valid_0's multi_error: 0.276172	valid_0's multi_logloss: 0.690848
training model for CV #4
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.874871
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.872043
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.868359
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.860886
[5]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.85471
[6]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.848927
[7]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.841004
[8]	valid_0's multi_error: 0.357739	valid_0's multi_logloss: 0.838533
[9]	valid_0's multi_error: 0.357097	valid_0's multi_logloss: 0.833546
[10]	valid_0's multi_error: 0.355812	valid_0's multi_logloss: 0.830824
[11]	valid_0's multi_error: 0.351317	valid_0's multi_loglos

[227]	valid_0's multi_error: 0.270392	valid_0's multi_logloss: 0.691468
[228]	valid_0's multi_error: 0.271676	valid_0's multi_logloss: 0.691431
[229]	valid_0's multi_error: 0.270392	valid_0's multi_logloss: 0.691225
[230]	valid_0's multi_error: 0.270392	valid_0's multi_logloss: 0.691348
training model for CV #5
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.874284
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.87111
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.867236
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.861014
[5]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.854171
[6]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.848454
[7]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.839302
[8]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.836511
[9]	valid_0's multi_error: 0.357739	valid_0's multi_logloss: 0.83188
[10]	valid_0's multi_error: 0.357097	valid_0's multi_loglos

...

[217]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.708231
[218]	valid_0's multi_error: 0.269453	valid_0's multi_logloss: 0.708137
[219]	valid_0's multi_error: 0.269453	valid_0's multi_logloss: 0.708227
[220]	valid_0's multi_error: 0.270096	valid_0's multi_logloss: 0.708332
[221]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.707955
[222]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.708222
[223]	valid_0's multi_error: 0.270096	valid_0's multi_logloss: 0.708378
[224]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.708393
[225]	valid_0's multi_error: 0.272026	valid_0's multi_logloss: 0.708484
[226]	valid_0's multi_error: 0.272026	valid_0's multi_logloss: 0.708131
[227]	valid_0's multi_error: 0.272026	valid_0's multi_logloss: 0.708345
[228]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.708374
[229]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.708411
[230]	valid_0's multi_error: 0.270096	valid_0's multi_logloss: 0.70

[216]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.680915
[217]	valid_0's multi_error: 0.273955	valid_0's multi_logloss: 0.680829
[218]	valid_0's multi_error: 0.273955	valid_0's multi_logloss: 0.680791
[219]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.680836
[220]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.680949
[221]	valid_0's multi_error: 0.272026	valid_0's multi_logloss: 0.680882
[222]	valid_0's multi_error: 0.272026	valid_0's multi_logloss: 0.680989
[223]	valid_0's multi_error: 0.273955	valid_0's multi_logloss: 0.680717
[224]	valid_0's multi_error: 0.273312	valid_0's multi_logloss: 0.680493
[225]	valid_0's multi_error: 0.273312	valid_0's multi_logloss: 0.680459
[226]	valid_0's multi_error: 0.273955	valid_0's multi_logloss: 0.680252
[227]	valid_0's multi_error: 0.273955	valid_0's multi_logloss: 0.680223
[228]	valid_0's multi_error: 0.273312	valid_0's multi_logloss: 0.680193
[229]	valid_0's multi_error: 0.273955	valid_0's multi_logloss: 0

[215]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.685446
[216]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.685821
[217]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.685875
[218]	valid_0's multi_error: 0.272669	valid_0's multi_logloss: 0.685985
[219]	valid_0's multi_error: 0.273312	valid_0's multi_logloss: 0.685933
[220]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.685897
[221]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.685681
[222]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.685571
[223]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.685777
[224]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.685969
[225]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.685996
[226]	valid_0's multi_error: 0.272026	valid_0's multi_logloss: 0.686195
[227]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.68638
[228]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.686

[213]	valid_0's multi_error: 0.258521	valid_0's multi_logloss: 0.666268
[214]	valid_0's multi_error: 0.258521	valid_0's multi_logloss: 0.666028
[215]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0.666189
[216]	valid_0's multi_error: 0.257235	valid_0's multi_logloss: 0.666205
[217]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0.666003
[218]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0.666148
[219]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0.666113
[220]	valid_0's multi_error: 0.257235	valid_0's multi_logloss: 0.665865
[221]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0.665686
[222]	valid_0's multi_error: 0.258521	valid_0's multi_logloss: 0.665572
[223]	valid_0's multi_error: 0.258521	valid_0's multi_logloss: 0.665436
[224]	valid_0's multi_error: 0.258521	valid_0's multi_logloss: 0.665541
[225]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0.665483
[226]	valid_0's multi_error: 0.257878	valid_0's multi_logloss: 0

[212]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.682021
[213]	valid_0's multi_error: 0.271383	valid_0's multi_logloss: 0.681566
[214]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.681483
[215]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.681294
[216]	valid_0's multi_error: 0.269453	valid_0's multi_logloss: 0.681353
[217]	valid_0's multi_error: 0.269453	valid_0's multi_logloss: 0.681193
[218]	valid_0's multi_error: 0.270096	valid_0's multi_logloss: 0.681343
[219]	valid_0's multi_error: 0.268167	valid_0's multi_logloss: 0.68101
[220]	valid_0's multi_error: 0.26881	valid_0's multi_logloss: 0.681014
[221]	valid_0's multi_error: 0.269453	valid_0's multi_logloss: 0.68077
[222]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.681045
[223]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.681067
[224]	valid_0's multi_error: 0.27074	valid_0's multi_logloss: 0.681159
[225]	valid_0's multi_error: 0.269453	valid_0's multi_logloss: 0.68119
[2

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/koko4/Desktop/credit/')
val_dir = Path('C:/Users/koko4/Desktop/credit/')

algo_name = 'lgb'
feature_name = '2021_j'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

### LGB_2

In [5]:
n_fold = 17
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr1.shape)

lgb_p_val2 = np.zeros((ftr1.shape[0], n_class))
lgb_p_tst2 = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    lgb_clf = lgb.LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=175, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 0)

    lgb_clf.fit(ftr1[i_trn], target1[i_trn],
            eval_set=[(ftr1[i_val], target1[i_val])])

    lgb_p_val2[i_val, :] = lgb_clf.predict_proba(ftr1[i_val])
    lgb_p_tst2 += lgb_clf.predict_proba(tst_ar1) / n_fold
    
print(f'{log_loss(target1, lgb_p_val2)}')
print(confusion_matrix(target1, np.argmax(lgb_p_val2, axis=1)))

(26457, 21)
training model for CV #1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.878837
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.874335
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.857246
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.853795
[5]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.850344
[6]	valid_0's multi_error: 0.356455	valid_0's multi_logloss: 0.841431
[7]	valid_0's multi_error

[112]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.68894
[113]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.688566
[114]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.687684
[115]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.687608
[116]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.68718
[117]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.686822
[118]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.686616
[119]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.685912
[120]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.685736
[121]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.685574
[122]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.685304
[123]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.685265
[124]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.684515
[125]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.6846

[59]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.728337
[60]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.727808
[61]	valid_0's multi_error: 0.280026	valid_0's multi_logloss: 0.727707
[62]	valid_0's multi_error: 0.28131	valid_0's multi_logloss: 0.726751
[63]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.726139
[64]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.726007
[65]	valid_0's multi_error: 0.279383	valid_0's multi_logloss: 0.725151
[66]	valid_0's multi_error: 0.278741	valid_0's multi_logloss: 0.724564
[67]	valid_0's multi_error: 0.280668	valid_0's multi_logloss: 0.723656
[68]	valid_0's multi_error: 0.280668	valid_0's multi_logloss: 0.722906
[69]	valid_0's multi_error: 0.277457	valid_0's multi_logloss: 0.721664
[70]	valid_0's multi_error: 0.277457	valid_0's multi_logloss: 0.720048
[71]	valid_0's multi_error: 0.278099	valid_0's multi_logloss: 0.71932
[72]	valid_0's multi_error: 0.277457	valid_0's multi_logloss: 0.719297
[73]	val

training model for CV #3
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001007 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1760
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.876766
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.872464
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.859245
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.855068
[5]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.852028
[6]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.844823
[7]	valid_0's multi_error: 0.358382	v

[116]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.691005
[117]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.690557
[118]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.690075
[119]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.68968
[120]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.689296
[121]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.689071
[122]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.688559
[123]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.688276
[124]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.688045
[125]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.687691
[126]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.687488
[127]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.686488
[128]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.685626
[129]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.68539

...

[34]	valid_0's multi_error: 0.298843	valid_0's multi_logloss: 0.755957
[35]	valid_0's multi_error: 0.293059	valid_0's multi_logloss: 0.752197
[36]	valid_0's multi_error: 0.293059	valid_0's multi_logloss: 0.751453
[37]	valid_0's multi_error: 0.291131	valid_0's multi_logloss: 0.749915
[38]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.748869
[39]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.747584
[40]	valid_0's multi_error: 0.291131	valid_0's multi_logloss: 0.744894
[41]	valid_0's multi_error: 0.289846	valid_0's multi_logloss: 0.74305
[42]	valid_0's multi_error: 0.289846	valid_0's multi_logloss: 0.742494
[43]	valid_0's multi_error: 0.28856	valid_0's multi_logloss: 0.741512
[44]	valid_0's multi_error: 0.287918	valid_0's multi_logloss: 0.740895
[45]	valid_0's multi_error: 0.287275	valid_0's multi_logloss: 0.74004
[46]	valid_0's multi_error: 0.287275	valid_0's multi_logloss: 0.738642
[47]	valid_0's multi_error: 0.284704	valid_0's multi_logloss: 0.736546
[48]	vali

[157]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680846
[158]	valid_0's multi_error: 0.266067	valid_0's multi_logloss: 0.681294
[159]	valid_0's multi_error: 0.26671	valid_0's multi_logloss: 0.681518
[160]	valid_0's multi_error: 0.266067	valid_0's multi_logloss: 0.681617
[161]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681084
[162]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680935
[163]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680716
[164]	valid_0's multi_error: 0.264139	valid_0's multi_logloss: 0.680627
[165]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680924
[166]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681029
[167]	valid_0's multi_error: 0.264781	valid_0's multi_logloss: 0.68111
[168]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681238
[169]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681051
[170]	valid_0's multi_error: 0.264781	valid_0's multi_logloss: 0.6

[96]	valid_0's multi_error: 0.280848	valid_0's multi_logloss: 0.704623
[97]	valid_0's multi_error: 0.279563	valid_0's multi_logloss: 0.704633
[98]	valid_0's multi_error: 0.279563	valid_0's multi_logloss: 0.704566
[99]	valid_0's multi_error: 0.278278	valid_0's multi_logloss: 0.703575
[100]	valid_0's multi_error: 0.27892	valid_0's multi_logloss: 0.702946
[101]	valid_0's multi_error: 0.278278	valid_0's multi_logloss: 0.702669
[102]	valid_0's multi_error: 0.27892	valid_0's multi_logloss: 0.702232
[103]	valid_0's multi_error: 0.27635	valid_0's multi_logloss: 0.701857
[104]	valid_0's multi_error: 0.27635	valid_0's multi_logloss: 0.701587
[105]	valid_0's multi_error: 0.277635	valid_0's multi_logloss: 0.701515
[106]	valid_0's multi_error: 0.276992	valid_0's multi_logloss: 0.701386
[107]	valid_0's multi_error: 0.275064	valid_0's multi_logloss: 0.701217
[108]	valid_0's multi_error: 0.275064	valid_0's multi_logloss: 0.700219
[109]	valid_0's multi_error: 0.275064	valid_0's multi_logloss: 0.700093


[36]	valid_0's multi_error: 0.298843	valid_0's multi_logloss: 0.759727
[37]	valid_0's multi_error: 0.300129	valid_0's multi_logloss: 0.758904
[38]	valid_0's multi_error: 0.299486	valid_0's multi_logloss: 0.756933
[39]	valid_0's multi_error: 0.299486	valid_0's multi_logloss: 0.755219
[40]	valid_0's multi_error: 0.296915	valid_0's multi_logloss: 0.752368
[41]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.750274
[42]	valid_0's multi_error: 0.294987	valid_0's multi_logloss: 0.749004
[43]	valid_0's multi_error: 0.294987	valid_0's multi_logloss: 0.747571
[44]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.746213
[45]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.745401
[46]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.744896
[47]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.742655
[48]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.741783
[49]	valid_0's multi_error: 0.293702	valid_0's multi_logloss: 0.739873
[50]	v

[159]	valid_0's multi_error: 0.272494	valid_0's multi_logloss: 0.704194
[160]	valid_0's multi_error: 0.271851	valid_0's multi_logloss: 0.704057
[161]	valid_0's multi_error: 0.270566	valid_0's multi_logloss: 0.703906
[162]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.704259
[163]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.704107
[164]	valid_0's multi_error: 0.268638	valid_0's multi_logloss: 0.703484
[165]	valid_0's multi_error: 0.26928	valid_0's multi_logloss: 0.703533
[166]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.703867
[167]	valid_0's multi_error: 0.270566	valid_0's multi_logloss: 0.703846
[168]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.704136
[169]	valid_0's multi_error: 0.271208	valid_0's multi_logloss: 0.704306
[170]	valid_0's multi_error: 0.272494	valid_0's multi_logloss: 0.704653
[171]	valid_0's multi_error: 0.271851	valid_0's multi_logloss: 0.704915
[172]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'lgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

### HGBM

In [5]:
# 그리드 서치
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='accuracy',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=8, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [48]:

# 데이터 분할
X_train,X_val,y_train,y_val=train_test_split(ftr1,target1,test_size=0.25,random_state=40)


# 모델 객체 생성
model = HistGradientBoostingClassifier()


paramgrid = {'max_iter' : [1000],
            'learning_rate' : [0.08, 0.09, 0.1, 0.11],
             'max_depth' : [15,17,18,20,22,24,26, 28, 30],
             'max_leaf_nodes' : [26,28,30, 31, 32, 33, 34],
             'random_state' : [2021]
            }


model, pred = algorithm_pipeline(X_train, X_val, y_train, y_val, model, 
                                 paramgrid, cv=10, scoring_fit='accuracy')


print(model.best_score_)
print(model.best_params_)

Fitting 10 folds for each of 252 candidates, totalling 2520 fits
0.712277413260746
{'learning_rate': 0.08, 'max_depth': 18, 'max_iter': 1000, 'max_leaf_nodes': 33, 'random_state': 2021}


In [7]:
n_fold = 10
print(ftr1.shape)
print(tst_ar1.shape)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=40)
 
hgb_p_val = np.zeros((ftr1.shape[0], n_class))
hgb_p_tst = np.zeros((tst_ar1.shape[0], n_class))
 
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    hgb_clf = HistGradientBoostingClassifier(max_iter=1000,
                                             learning_rate = 0.08,
                                                verbose=True,
                                             max_leaf_nodes = 33,
                                             max_depth = 18,
                                                random_state=2021
                                                )
    hgb_clf.fit(ftr1[i_trn], target1[i_trn])
    hgb_p_val[i_val, :] = hgb_clf.predict_proba(ftr1[i_val])
    hgb_p_tst += hgb_clf.predict_proba(tst_ar1) / n_fold
    
print(f'{log_loss(target1, hgb_p_val)}')
print(confusion_matrix(target1, np.argmax(hgb_p_val, axis=1)))

(26457, 21)
(10000, 21)
training model for CV #1
Binning 0.004 GB of training data: 0.023 s
Binning 0.000 GB of validation data: 0.001 s
Fitting gradient boosted rounds:
[1/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.85811, val loss: 0.85893, in 0.027s
[2/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.84105, val loss: 0.84243, in 0.029s
[3/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.82802, val loss: 0.83105, in 0.026s
[4/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.81728, val loss: 0.82308, in 0.027s
[5/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.80813, val loss: 0.81526, in 0.028s
[6/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.80043, val loss: 0.80971, in 0.027s
[7/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.79320, val loss: 0.80414, in 0.027s
[8/1000] 3 trees, 99 leaves (33 on avg), max depth = 10, train loss: 0.78651, val

[76/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.62432, val loss: 0.73233, in 0.026s
[77/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.62252, val loss: 0.73198, in 0.027s
[78/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.62069, val loss: 0.73148, in 0.028s
[79/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.61908, val loss: 0.73125, in 0.027s
[80/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.61776, val loss: 0.73067, in 0.027s
[81/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.61622, val loss: 0.73020, in 0.028s
[82/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.61490, val loss: 0.72988, in 0.024s
[83/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.61310, val loss: 0.72927, in 0.027s
[84/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.61184, val loss: 0.72941, in 0.027s
[85/1000] 3 trees, 99 leaves

[152/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.52599, val loss: 0.71603, in 0.026s
[153/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.52506, val loss: 0.71587, in 0.025s
[154/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.52392, val loss: 0.71545, in 0.025s
[155/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.52281, val loss: 0.71536, in 0.025s
[156/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.52156, val loss: 0.71510, in 0.024s
[157/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.52067, val loss: 0.71476, in 0.025s
[158/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.51965, val loss: 0.71454, in 0.026s
[159/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.51841, val loss: 0.71436, in 0.026s
[160/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.51706, val loss: 0.71396, in 0.025s
[161/1000] 3 trees,

[39/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.68707, val loss: 0.75469, in 0.025s
[40/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.68496, val loss: 0.75405, in 0.024s
[41/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.68320, val loss: 0.75305, in 0.026s
[42/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.68094, val loss: 0.75238, in 0.025s
[43/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.67848, val loss: 0.75212, in 0.026s
[44/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.67643, val loss: 0.75166, in 0.025s
[45/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.67451, val loss: 0.75077, in 0.025s
[46/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.67257, val loss: 0.75040, in 0.025s
[47/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.67024, val loss: 0.74995, in 0.025s
[48/1000] 3 trees, 99 leaves

[115/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.56629, val loss: 0.72631, in 0.024s
[116/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.56494, val loss: 0.72588, in 0.024s
[117/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.56384, val loss: 0.72595, in 0.025s
[118/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.56293, val loss: 0.72612, in 0.023s
[119/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.56156, val loss: 0.72595, in 0.026s
[120/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.56044, val loss: 0.72565, in 0.025s
[121/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.55912, val loss: 0.72515, in 0.025s
[122/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.55796, val loss: 0.72526, in 0.025s
[123/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.55666, val loss: 0.72480, in 0.025s
[124/1000] 3 trees,

...

[159/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.51488, val loss: 0.72837, in 0.029s
[160/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.51378, val loss: 0.72852, in 0.027s
[161/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.51271, val loss: 0.72857, in 0.027s
[162/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.51150, val loss: 0.72836, in 0.027s
[163/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.51073, val loss: 0.72824, in 0.027s
[164/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.50977, val loss: 0.72809, in 0.028s
[165/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.50847, val loss: 0.72828, in 0.026s
[166/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.50744, val loss: 0.72811, in 0.023s
[167/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.50630, val loss: 0.72780, in 0.026s
[168/1000] 3 trees,

[38/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.69078, val loss: 0.75585, in 0.034s
[39/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.68805, val loss: 0.75462, in 0.032s
[40/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.68612, val loss: 0.75458, in 0.030s
[41/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.68395, val loss: 0.75409, in 0.029s
[42/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.68227, val loss: 0.75343, in 0.032s
[43/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.67985, val loss: 0.75271, in 0.030s
[44/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.67759, val loss: 0.75172, in 0.029s
[45/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.67601, val loss: 0.75161, in 0.027s
[46/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.67409, val loss: 0.75134, in 0.028s
[47/1000] 3 trees, 99 leaves

[36/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.69450, val loss: 0.76149, in 0.025s
[37/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.69268, val loss: 0.76058, in 0.024s
[38/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.69023, val loss: 0.76019, in 0.026s
[39/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.68785, val loss: 0.75892, in 0.027s
[40/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.68598, val loss: 0.75813, in 0.025s
[41/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.68360, val loss: 0.75721, in 0.026s
[42/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.68149, val loss: 0.75635, in 0.024s
[43/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.67961, val loss: 0.75546, in 0.027s
[44/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.67739, val loss: 0.75461, in 0.025s
[45/1000] 3 trees, 99 leaves

[112/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.56966, val loss: 0.71971, in 0.026s
[113/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.56862, val loss: 0.71959, in 0.025s
[114/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.56710, val loss: 0.71903, in 0.026s
[115/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.56597, val loss: 0.71896, in 0.024s
[116/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.56478, val loss: 0.71882, in 0.024s
[117/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.56351, val loss: 0.71822, in 0.025s
[118/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.56210, val loss: 0.71718, in 0.025s
[119/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.56086, val loss: 0.71664, in 0.024s
[120/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.55984, val loss: 0.71639, in 0.024s
[121/1000] 3 trees,

[188/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48701, val loss: 0.70543, in 0.025s
[189/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48587, val loss: 0.70465, in 0.029s
[190/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.48502, val loss: 0.70442, in 0.027s
[191/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.48414, val loss: 0.70454, in 0.025s
[192/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.48314, val loss: 0.70444, in 0.028s
[193/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.48205, val loss: 0.70420, in 0.027s
[194/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48124, val loss: 0.70429, in 0.026s
[195/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48030, val loss: 0.70383, in 0.028s
[196/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.47958, val loss: 0.70371, in 0.027s
[197/1000] 3 trees,

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'hgbm'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, hgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, hgb_p_tst, fmt='%.6f', delimiter=',')

### xgboost

In [ ]:
n_fold = 17

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr1.shape)

xgb_p_val = np.zeros((ftr1.shape[0], n_class))
xgb_p_tst = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    xgb_clf = xgb.XGBClassifier(max_depth=10,
                                colsample_bytree=0.5,
                                colsample_bylevel=0.8,
                                colsample_bynode=0.5,
                                min_child_weight=0.5,
                                gamma=0,
                                max_delta_step=0,
                                subsample=0.9,
                                reg_alpha=0.09,
                                reg_lambda=0.6,
                                n_estimators=50,
                               random_state = 2021)

    xgb_clf.fit(ftr1[i_trn], target1[i_trn],
            eval_set=[(ftr1[i_val], target1[i_val])])
    
    xgb_p_val[i_val, :] = xgb_clf.predict_proba(ftr1[i_val])
    xgb_p_tst += xgb_clf.predict_proba(tst_ar1) / n_fold


print(f'{log_loss(target1, xgb_p_val)}')
print(confusion_matrix(target1, np.argmax(xgb_p_val, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'xgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, xgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, xgb_p_tst, fmt='%.6f', delimiter=',')

### catboost

In [5]:
n_fold = 10
target1 = cat_credit.values

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2020)
 
print(ftr1.shape)
 
cb_p_val = np.zeros((ftr1.shape[0], n_class))
cb_p_tst = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    cb_clf = CatBoostClassifier(depth=8,
                                iterations=2000,
                                rsm=0.7,
                                ignored_features=5,
                                learning_rate=0.04,
                                l2_leaf_reg=1,
                               random_state = 2021)
    
    cb_clf.fit(ftr1[i_trn], target1[i_trn],
    eval_set=[(ftr1[i_val], target1[i_val])])
    
    cb_p_val[i_val, :] = cb_clf.predict_proba(ftr1[i_val])
    cb_p_tst += cb_clf.predict_proba(tst_ar1) / n_fold

    
print(f'{log_loss(target1, cb_p_val)}')
print(confusion_matrix(target1, np.argmax(cb_p_val, axis=1)))

(26457, 21)
training model for CV #1
0:	learn: 1.0755413	test: 1.0763213	best: 1.0763213 (0)	total: 164ms	remaining: 5m 28s
1:	learn: 1.0543992	test: 1.0554459	best: 1.0554459 (1)	total: 168ms	remaining: 2m 47s
2:	learn: 1.0350521	test: 1.0365880	best: 1.0365880 (2)	total: 183ms	remaining: 2m 1s
3:	learn: 1.0174846	test: 1.0195347	best: 1.0195347 (3)	total: 199ms	remaining: 1m 39s
4:	learn: 1.0012704	test: 1.0040051	best: 1.0040051 (4)	total: 215ms	remaining: 1m 25s
5:	learn: 0.9864137	test: 0.9896757	best: 0.9896757 (5)	total: 231ms	remaining: 1m 16s
6:	learn: 0.9731741	test: 0.9767520	best: 0.9767520 (6)	total: 234ms	remaining: 1m 6s
7:	learn: 0.9606530	test: 0.9647697	best: 0.9647697 (7)	total: 250ms	remaining: 1m 2s
8:	learn: 0.9491156	test: 0.9536666	best: 0.9536666 (8)	total: 267ms	remaining: 59s
9:	learn: 0.9384198	test: 0.9434156	best: 0.9434156 (9)	total: 282ms	remaining: 56.2s
10:	learn: 0.9285657	test: 0.9338455	best: 0.9338455 (10)	total: 298ms	remaining: 53.8s
11:	learn: 0

101:	learn: 0.7616338	test: 0.7971339	best: 0.7971339 (101)	total: 1.8s	remaining: 33.5s
102:	learn: 0.7610237	test: 0.7969887	best: 0.7969887 (102)	total: 1.82s	remaining: 33.5s
103:	learn: 0.7606811	test: 0.7968310	best: 0.7968310 (103)	total: 1.84s	remaining: 33.5s
104:	learn: 0.7600588	test: 0.7965015	best: 0.7965015 (104)	total: 1.85s	remaining: 33.4s
105:	learn: 0.7595703	test: 0.7961770	best: 0.7961770 (105)	total: 1.87s	remaining: 33.4s
106:	learn: 0.7591474	test: 0.7959713	best: 0.7959713 (106)	total: 1.89s	remaining: 33.4s
107:	learn: 0.7586018	test: 0.7956104	best: 0.7956104 (107)	total: 1.9s	remaining: 33.3s
108:	learn: 0.7578404	test: 0.7951522	best: 0.7951522 (108)	total: 1.92s	remaining: 33.3s
109:	learn: 0.7574102	test: 0.7948905	best: 0.7948905 (109)	total: 1.94s	remaining: 33.3s
110:	learn: 0.7568628	test: 0.7947434	best: 0.7947434 (110)	total: 1.95s	remaining: 33.2s
111:	learn: 0.7559122	test: 0.7943793	best: 0.7943793 (111)	total: 1.97s	remaining: 33.2s
112:	learn: 

200:	learn: 0.7155973	test: 0.7769310	best: 0.7769310 (200)	total: 3.41s	remaining: 30.5s
201:	learn: 0.7151097	test: 0.7766835	best: 0.7766835 (201)	total: 3.43s	remaining: 30.5s
202:	learn: 0.7145650	test: 0.7764140	best: 0.7764140 (202)	total: 3.45s	remaining: 30.5s
203:	learn: 0.7142112	test: 0.7762651	best: 0.7762651 (203)	total: 3.46s	remaining: 30.5s
204:	learn: 0.7137569	test: 0.7759609	best: 0.7759609 (204)	total: 3.48s	remaining: 30.5s
205:	learn: 0.7134517	test: 0.7759054	best: 0.7759054 (205)	total: 3.5s	remaining: 30.5s
206:	learn: 0.7132348	test: 0.7758219	best: 0.7758219 (206)	total: 3.52s	remaining: 30.4s
207:	learn: 0.7129482	test: 0.7757604	best: 0.7757604 (207)	total: 3.53s	remaining: 30.4s
208:	learn: 0.7127888	test: 0.7756757	best: 0.7756757 (208)	total: 3.55s	remaining: 30.4s
209:	learn: 0.7124046	test: 0.7756443	best: 0.7756443 (209)	total: 3.57s	remaining: 30.4s
210:	learn: 0.7122211	test: 0.7755961	best: 0.7755961 (210)	total: 3.58s	remaining: 30.4s
211:	learn:

297:	learn: 0.6795708	test: 0.7639500	best: 0.7639500 (297)	total: 5.01s	remaining: 28.6s
298:	learn: 0.6791452	test: 0.7636963	best: 0.7636963 (298)	total: 5.03s	remaining: 28.6s
299:	learn: 0.6789372	test: 0.7636593	best: 0.7636593 (299)	total: 5.05s	remaining: 28.6s
300:	learn: 0.6784221	test: 0.7633739	best: 0.7633739 (300)	total: 5.06s	remaining: 28.6s
301:	learn: 0.6780664	test: 0.7632445	best: 0.7632445 (301)	total: 5.08s	remaining: 28.6s
302:	learn: 0.6777805	test: 0.7631776	best: 0.7631776 (302)	total: 5.09s	remaining: 28.5s
303:	learn: 0.6776185	test: 0.7632294	best: 0.7631776 (302)	total: 5.11s	remaining: 28.5s
304:	learn: 0.6774288	test: 0.7631575	best: 0.7631575 (304)	total: 5.13s	remaining: 28.5s
305:	learn: 0.6770796	test: 0.7630539	best: 0.7630539 (305)	total: 5.14s	remaining: 28.5s
306:	learn: 0.6766324	test: 0.7628832	best: 0.7628832 (306)	total: 5.16s	remaining: 28.4s
307:	learn: 0.6760949	test: 0.7626817	best: 0.7626817 (307)	total: 5.17s	remaining: 28.4s
308:	learn

397:	learn: 0.6456454	test: 0.7523329	best: 0.7523329 (397)	total: 6.63s	remaining: 26.7s
398:	learn: 0.6453769	test: 0.7523386	best: 0.7523329 (397)	total: 6.64s	remaining: 26.7s
399:	learn: 0.6451891	test: 0.7522747	best: 0.7522747 (399)	total: 6.66s	remaining: 26.6s
400:	learn: 0.6445615	test: 0.7518469	best: 0.7518469 (400)	total: 6.67s	remaining: 26.6s
401:	learn: 0.6441253	test: 0.7517059	best: 0.7517059 (401)	total: 6.69s	remaining: 26.6s
402:	learn: 0.6435950	test: 0.7515952	best: 0.7515952 (402)	total: 6.71s	remaining: 26.6s
403:	learn: 0.6432459	test: 0.7514176	best: 0.7514176 (403)	total: 6.73s	remaining: 26.6s
404:	learn: 0.6429033	test: 0.7513137	best: 0.7513137 (404)	total: 6.75s	remaining: 26.6s
405:	learn: 0.6426476	test: 0.7512327	best: 0.7512327 (405)	total: 6.76s	remaining: 26.6s
406:	learn: 0.6424841	test: 0.7511614	best: 0.7511614 (406)	total: 6.78s	remaining: 26.5s
407:	learn: 0.6422779	test: 0.7511700	best: 0.7511614 (406)	total: 6.8s	remaining: 26.5s
408:	learn:

...

1558:	learn: 0.4347457	test: 0.6883735	best: 0.6883030 (1543)	total: 23.7s	remaining: 6.71s
1559:	learn: 0.4346214	test: 0.6883305	best: 0.6883030 (1543)	total: 23.7s	remaining: 6.69s
1560:	learn: 0.4345351	test: 0.6883172	best: 0.6883030 (1543)	total: 23.7s	remaining: 6.67s
1561:	learn: 0.4344395	test: 0.6883390	best: 0.6883030 (1543)	total: 23.7s	remaining: 6.66s
1562:	learn: 0.4342755	test: 0.6883358	best: 0.6883030 (1543)	total: 23.8s	remaining: 6.64s
1563:	learn: 0.4341298	test: 0.6883390	best: 0.6883030 (1543)	total: 23.8s	remaining: 6.63s
1564:	learn: 0.4340459	test: 0.6883363	best: 0.6883030 (1543)	total: 23.8s	remaining: 6.61s
1565:	learn: 0.4339561	test: 0.6883706	best: 0.6883030 (1543)	total: 23.8s	remaining: 6.6s
1566:	learn: 0.4338376	test: 0.6883797	best: 0.6883030 (1543)	total: 23.8s	remaining: 6.58s
1567:	learn: 0.4337766	test: 0.6884285	best: 0.6883030 (1543)	total: 23.8s	remaining: 6.57s
1568:	learn: 0.4336234	test: 0.6883861	best: 0.6883030 (1543)	total: 23.9s	remain

1651:	learn: 0.4237931	test: 0.6880371	best: 0.6880193 (1634)	total: 25.1s	remaining: 5.29s
1652:	learn: 0.4236482	test: 0.6879489	best: 0.6879489 (1652)	total: 25.1s	remaining: 5.28s
1653:	learn: 0.4235436	test: 0.6879242	best: 0.6879242 (1653)	total: 25.2s	remaining: 5.26s
1654:	learn: 0.4234111	test: 0.6878901	best: 0.6878901 (1654)	total: 25.2s	remaining: 5.25s
1655:	learn: 0.4233483	test: 0.6878569	best: 0.6878569 (1655)	total: 25.2s	remaining: 5.23s
1656:	learn: 0.4232840	test: 0.6878578	best: 0.6878569 (1655)	total: 25.2s	remaining: 5.22s
1657:	learn: 0.4231129	test: 0.6878419	best: 0.6878419 (1657)	total: 25.2s	remaining: 5.2s
1658:	learn: 0.4230360	test: 0.6878367	best: 0.6878367 (1658)	total: 25.2s	remaining: 5.19s
1659:	learn: 0.4229896	test: 0.6877907	best: 0.6877907 (1659)	total: 25.3s	remaining: 5.17s
1660:	learn: 0.4228834	test: 0.6878138	best: 0.6877907 (1659)	total: 25.3s	remaining: 5.16s
1661:	learn: 0.4227898	test: 0.6878548	best: 0.6877907 (1659)	total: 25.3s	remain

1743:	learn: 0.4132587	test: 0.6875709	best: 0.6871664 (1697)	total: 26.5s	remaining: 3.89s
1744:	learn: 0.4131241	test: 0.6876054	best: 0.6871664 (1697)	total: 26.5s	remaining: 3.88s
1745:	learn: 0.4130230	test: 0.6875858	best: 0.6871664 (1697)	total: 26.5s	remaining: 3.86s
1746:	learn: 0.4129356	test: 0.6875886	best: 0.6871664 (1697)	total: 26.6s	remaining: 3.85s
1747:	learn: 0.4128434	test: 0.6875922	best: 0.6871664 (1697)	total: 26.6s	remaining: 3.83s
1748:	learn: 0.4127228	test: 0.6876866	best: 0.6871664 (1697)	total: 26.6s	remaining: 3.82s
1749:	learn: 0.4126579	test: 0.6876442	best: 0.6871664 (1697)	total: 26.6s	remaining: 3.8s
1750:	learn: 0.4125739	test: 0.6876851	best: 0.6871664 (1697)	total: 26.6s	remaining: 3.79s
1751:	learn: 0.4124667	test: 0.6876882	best: 0.6871664 (1697)	total: 26.6s	remaining: 3.77s
1752:	learn: 0.4123494	test: 0.6876906	best: 0.6871664 (1697)	total: 26.7s	remaining: 3.76s
1753:	learn: 0.4122686	test: 0.6876857	best: 0.6871664 (1697)	total: 26.7s	remain

1835:	learn: 0.4037984	test: 0.6871449	best: 0.6871449 (1835)	total: 27.9s	remaining: 2.49s
1836:	learn: 0.4036647	test: 0.6871442	best: 0.6871442 (1836)	total: 27.9s	remaining: 2.48s
1837:	learn: 0.4035836	test: 0.6871249	best: 0.6871249 (1837)	total: 28s	remaining: 2.46s
1838:	learn: 0.4034776	test: 0.6871511	best: 0.6871249 (1837)	total: 28s	remaining: 2.45s
1839:	learn: 0.4034014	test: 0.6872027	best: 0.6871249 (1837)	total: 28s	remaining: 2.43s
1840:	learn: 0.4032932	test: 0.6872489	best: 0.6871249 (1837)	total: 28s	remaining: 2.42s
1841:	learn: 0.4031815	test: 0.6872454	best: 0.6871249 (1837)	total: 28s	remaining: 2.4s
1842:	learn: 0.4030648	test: 0.6872373	best: 0.6871249 (1837)	total: 28s	remaining: 2.39s
1843:	learn: 0.4029935	test: 0.6872644	best: 0.6871249 (1837)	total: 28.1s	remaining: 2.37s
1844:	learn: 0.4028344	test: 0.6872870	best: 0.6871249 (1837)	total: 28.1s	remaining: 2.36s
1845:	learn: 0.4026588	test: 0.6873243	best: 0.6871249 (1837)	total: 28.1s	remaining: 2.34s
1

1928:	learn: 0.3948034	test: 0.6875044	best: 0.6871249 (1837)	total: 29.3s	remaining: 1.08s
1929:	learn: 0.3947059	test: 0.6875302	best: 0.6871249 (1837)	total: 29.3s	remaining: 1.06s
1930:	learn: 0.3946068	test: 0.6874707	best: 0.6871249 (1837)	total: 29.4s	remaining: 1.05s
1931:	learn: 0.3945423	test: 0.6874680	best: 0.6871249 (1837)	total: 29.4s	remaining: 1.03s
1932:	learn: 0.3944177	test: 0.6874567	best: 0.6871249 (1837)	total: 29.4s	remaining: 1.02s
1933:	learn: 0.3942649	test: 0.6874991	best: 0.6871249 (1837)	total: 29.4s	remaining: 1s
1934:	learn: 0.3941471	test: 0.6874902	best: 0.6871249 (1837)	total: 29.4s	remaining: 988ms
1935:	learn: 0.3940336	test: 0.6875248	best: 0.6871249 (1837)	total: 29.4s	remaining: 973ms
1936:	learn: 0.3938990	test: 0.6875151	best: 0.6871249 (1837)	total: 29.4s	remaining: 958ms
1937:	learn: 0.3938201	test: 0.6873844	best: 0.6871249 (1837)	total: 29.5s	remaining: 942ms
1938:	learn: 0.3937645	test: 0.6873965	best: 0.6871249 (1837)	total: 29.5s	remainin

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'cat'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, cb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, cb_p_tst, fmt='%.6f', delimiter=',')

### random forest

In [27]:
n_fold = 10

print(ftr2.shape)
print(tst_ar2.shape)

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr2.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar2.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr2, target2), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 7000, 
                                random_state=2021,
                                max_features = 3,
                                max_depth = 28,
                                min_samples_split = 8,
                                n_jobs=4)
    
    rf_clf.fit(ftr2[i_trn], target2[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr2[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar2) / n_fold
    
print(f'{log_loss(target2, rf_p_val)}')
print(confusion_matrix(target2, np.argmax(rf_p_val, axis=1)))

(26457, 22)
(10000, 22)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5
training model for CV #6
training model for CV #7
training model for CV #8
training model for CV #9
training model for CV #10
0.6856012218673924
[[  650   506  2066]
 [  128  2866  3273]
 [  305   768 15895]]%


In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'rf'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, rf_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, rf_p_tst, fmt='%.6f', delimiter=',')

### extra tree

In [14]:
n_fold = 10

print(ftr2.shape)
print(tst_ar2.shape)

cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 42)

ext_p_val = np.zeros((ftr2.shape[0], n_class))
ext_p_tst = np.zeros((tst_ar2.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr2, target2), 1):
    print(f'training model for CV #{i}')
    ext_clf = ExtraTreesClassifier(
                                   max_depth = 30, 
                                   max_features = 4, 
                                   min_samples_split = 9, 
                                   n_estimators = 7000, 
                                   random_state = 2021,
                                  n_jobs = 4)

    ext_clf.fit(ftr2[i_trn], target2[i_trn])
    ext_p_val[i_val, :] = ext_clf.predict_proba(ftr2[i_val])
    ext_p_tst += ext_clf.predict_proba(tst_ar2) / n_fold

print(f'{log_loss(target2, ext_p_val)}')
print(confusion_matrix(target2, np.argmax(ext_p_val, axis=1)))

(26457, 22)
(10000, 22)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5
training model for CV #6
training model for CV #7
training model for CV #8
training model for CV #9
training model for CV #10
0.7133101117901619
[[  662   441  2119]
 [  150  2618  3499]
 [  315   887 15766]]


In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'extra'
feature_name = '0.711'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, ext_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, ext_p_tst, fmt='%.6f', delimiter=',')

### stacking

In [2]:
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

model_names = ['rf_2021',
               'xgb_2021',
               'lgb_2021',
                 'extra_2021',
                'cat_2021',
               'lgb_2021_1',
                'hgbm_2021'
              ]


stk_trn = []
stk_tst = []
feature_names = []
for model in model_names:
    stk_trn.append(np.loadtxt(val_dir / f'{model}.val.csv', delimiter=','))
    stk_tst.append(np.loadtxt(tst_dir / f'{model}.tst.csv', delimiter=','))
    feature_names += [f'{model}_credit0', f'{model}_credit1', f'{model}_credit2']
    
stk_trn = np.hstack(stk_trn)
stk_tst = np.hstack(stk_tst)
feature_names

['rf_2021_credit0',
 'rf_2021_credit1',
 'rf_2021_credit2',
 'xgb_2021_credit0',
 'xgb_2021_credit1',
 'xgb_2021_credit2',
 'lgb_2021_credit0',
 'lgb_2021_credit1',
 'lgb_2021_credit2',
 'extra_2021_credit0',
 'extra_2021_credit1',
 'extra_2021_credit2',
 'cat_2021_credit0',
 'cat_2021_credit1',
 'cat_2021_credit2',
 'lgb_2021_j_credit0',
 'lgb_2021_j_credit1',
 'lgb_2021_j_credit2',
 'hgbm_2021_credit0',
 'hgbm_2021_credit1',
 'hgbm_2021_credit2']

In [3]:
stk_trn.shape
stk_tst.shape

(10000, 21)

In [4]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

n_fold = 13
seed = 2021
n_class = 3

target=trn['credit'].values
y=target.copy()

In [5]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2021)

p_trn =np.zeros((stk_trn.shape[0], 3))
p_val = np.zeros((stk_trn.shape[0], 3))
p_tst = np.zeros((stk_tst.shape[0], 3))
for i, (i_trn, i_val) in enumerate(cv.split(stk_trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(boosting_type='gbdt',
                             max_depth=2,
                             num_leaves=10,
                             colsample_bytree=0.9,
                             n_estimators=110, 
                             min_child_samples=24,
                             subsample=0.5,
                             subsample_freq=1,
                             learning_rate=0.1,
                             random_state=1557,
                             verbose = 50)
    
    clf.fit(stk_trn[i_trn], y[i_trn],
            eval_set=[(stk_trn[i_val], y[i_val])],
            eval_metric='logloss',
            verbose=True)

    p_val[i_val, :] = clf.predict_proba(stk_trn[i_val])
    p_tst += clf.predict_proba(stk_tst) / n_fold
    
print(clf)

print(f'{log_loss(target, p_val)}')
print(f'{confusion_matrix(target, np.argmax(p_val, axis=1))}%')

training model for CV #1
[1]	valid_0's multi_logloss: 0.846845
[2]	valid_0's multi_logloss: 0.819668
[3]	valid_0's multi_logloss: 0.797396
[4]	valid_0's multi_logloss: 0.778556
[5]	valid_0's multi_logloss: 0.762919
[6]	valid_0's multi_logloss: 0.749518
[7]	valid_0's multi_logloss: 0.738242
[8]	valid_0's multi_logloss: 0.728285
[9]	valid_0's multi_logloss: 0.719778
[10]	valid_0's multi_logloss: 0.712382
[11]	valid_0's multi_logloss: 0.705745
[12]	valid_0's multi_logloss: 0.700172
[13]	valid_0's multi_logloss: 0.694939
[14]	valid_0's multi_logloss: 0.690616
[15]	valid_0's multi_logloss: 0.686653
[16]	valid_0's multi_logloss: 0.683255
[17]	valid_0's multi_logloss: 0.680349
[18]	valid_0's multi_logloss: 0.677922
[19]	valid_0's multi_logloss: 0.675566
[20]	valid_0's multi_logloss: 0.673537
[21]	valid_0's multi_logloss: 0.671628
[22]	valid_0's multi_logloss: 0.670011
[23]	valid_0's multi_logloss: 0.66859
[24]	valid_0's multi_logloss: 0.667242
[25]	valid_0's multi_logloss: 0.666077
[26]	valid

[89]	valid_0's multi_logloss: 0.651999
[90]	valid_0's multi_logloss: 0.652146
[91]	valid_0's multi_logloss: 0.652092
[92]	valid_0's multi_logloss: 0.652209
[93]	valid_0's multi_logloss: 0.65214
[94]	valid_0's multi_logloss: 0.652184
[95]	valid_0's multi_logloss: 0.652163
[96]	valid_0's multi_logloss: 0.652072
[97]	valid_0's multi_logloss: 0.651843
[98]	valid_0's multi_logloss: 0.651929
[99]	valid_0's multi_logloss: 0.651941
[100]	valid_0's multi_logloss: 0.651811
[101]	valid_0's multi_logloss: 0.651919
[102]	valid_0's multi_logloss: 0.652153
[103]	valid_0's multi_logloss: 0.651971
[104]	valid_0's multi_logloss: 0.65191
[105]	valid_0's multi_logloss: 0.651967
[106]	valid_0's multi_logloss: 0.65204
[107]	valid_0's multi_logloss: 0.6521
[108]	valid_0's multi_logloss: 0.651962
[109]	valid_0's multi_logloss: 0.651994
[110]	valid_0's multi_logloss: 0.651849
training model for CV #5
[1]	valid_0's multi_logloss: 0.850234
[2]	valid_0's multi_logloss: 0.824886
[3]	valid_0's multi_logloss: 0.8041

[67]	valid_0's multi_logloss: 0.677852
[68]	valid_0's multi_logloss: 0.677856
[69]	valid_0's multi_logloss: 0.677971
[70]	valid_0's multi_logloss: 0.677904
[71]	valid_0's multi_logloss: 0.67804
[72]	valid_0's multi_logloss: 0.678069
[73]	valid_0's multi_logloss: 0.678137
[74]	valid_0's multi_logloss: 0.678114
[75]	valid_0's multi_logloss: 0.678049
[76]	valid_0's multi_logloss: 0.67794
[77]	valid_0's multi_logloss: 0.677833
[78]	valid_0's multi_logloss: 0.67791
[79]	valid_0's multi_logloss: 0.677775
[80]	valid_0's multi_logloss: 0.677978
[81]	valid_0's multi_logloss: 0.677904
[82]	valid_0's multi_logloss: 0.67776
[83]	valid_0's multi_logloss: 0.677781
[84]	valid_0's multi_logloss: 0.67788
[85]	valid_0's multi_logloss: 0.678035
[86]	valid_0's multi_logloss: 0.677998
[87]	valid_0's multi_logloss: 0.677953
[88]	valid_0's multi_logloss: 0.678006
[89]	valid_0's multi_logloss: 0.677702
[90]	valid_0's multi_logloss: 0.677761
[91]	valid_0's multi_logloss: 0.677822
[92]	valid_0's multi_logloss: 

[45]	valid_0's multi_logloss: 0.665966
[46]	valid_0's multi_logloss: 0.665646
[47]	valid_0's multi_logloss: 0.665591
[48]	valid_0's multi_logloss: 0.665584
[49]	valid_0's multi_logloss: 0.665414
[50]	valid_0's multi_logloss: 0.665105
[51]	valid_0's multi_logloss: 0.664903
[52]	valid_0's multi_logloss: 0.66468
[53]	valid_0's multi_logloss: 0.664572
[54]	valid_0's multi_logloss: 0.664491
[55]	valid_0's multi_logloss: 0.664278
[56]	valid_0's multi_logloss: 0.664268
[57]	valid_0's multi_logloss: 0.664267
[58]	valid_0's multi_logloss: 0.664218
[59]	valid_0's multi_logloss: 0.664203
[60]	valid_0's multi_logloss: 0.664044
[61]	valid_0's multi_logloss: 0.663973
[62]	valid_0's multi_logloss: 0.663987
[63]	valid_0's multi_logloss: 0.66386
[64]	valid_0's multi_logloss: 0.66374
[65]	valid_0's multi_logloss: 0.66376
[66]	valid_0's multi_logloss: 0.663797
[67]	valid_0's multi_logloss: 0.663773
[68]	valid_0's multi_logloss: 0.663743
[69]	valid_0's multi_logloss: 0.663511
[70]	valid_0's multi_logloss:

In [ ]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub[sub.columns] = p_tst
sub.to_csv('stack_final.csv')